**классификатор текстов LSTM на Keras+TensorFlow**

Евгений Борисов <borisov.e@solarl.ru>

In [1]:
# https://habr.com/ru/company/dca/blog/274027/
# http://neuro.compute.dtu.dk/wiki/Sentiment_analysis#Corpora
# http://help.sentiment140.com/for-students/
# http://study.mokoron.com

In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 200  
import re
import gzip

In [3]:
def pp(d): return "{:,.0f}".format(d).replace(",", " ")
def ppr(d): print('записей:', pp(len(d)) )  

---

In [4]:
ff = ['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount','unk']

In [5]:
neg = pd.read_csv('../data/text/twit/negative.csv.gz',sep=';',header=None)
ppr(neg)
neg.columns = ff

записей: 111 923


In [6]:
pos = pd.read_csv('../data/text/twit/positive.csv.gz',sep=';')
ppr(pos)
pos.columns = ff

записей: 114 910


In [7]:
data = pd.concat([pos,neg],sort=False)[['id','ttext', 'ttype']]
ppr(data)

записей: 226 833


In [8]:
data.sample(10)

,id,ttext,ttype
111046,411199185225277440,но надеюсь что мы выйграем)потомучто наш класс молодец)мы дахера чего придумали лахачного) у нас будет офигенная песня и конь)УДАЧИ НАМ^^,1
102790,411106081490759680,"Спокойной ночи,лента с:\nУдачи в завтрашнем дне)\nОдевайтесь теплее,на улице становится все холоднее и холоднее) http://t.co/7VWYGEivF8",1
45464,409985975763218432,"Сегодня день мультиков...посмотрела ""Храбрая сердцем"" и ""Семейка крудс""-обалденные мультики)",1
81849,410760619806040064,@Juli_Suzun фильм красив!) а он восхитительно играет на скрипке.,1
92393,422085649865658369,Не надо я буду скучать(((( — Печаль. http://t.co/SqGkDfx6cb,-1
60186,410276397824106497,А у нас урок по химии всегда прикольно проходит ДЗ не спрашивают)) сидим о политике разговариваем)) #haha,1
31555,409656949702414336,RT @katkostuychenko: @Miller_Anna_ мимимимимимим....спасибочки)))))),1
9140,409195868944732160,@alinochka_smile ну вот и хорошо) замечательно :3 \nприедешь будем здесь с тобой тусить\nа так- то ты была в мск??,1
13499,410825154164699137,Я жопорукий долбоеб! :( Я уже залил 200 метров на этот самый народ диск и нажал Esc,-1
58090,410128775230533632,"Юля, неужели ты думаешь, что я до такой степени наивная?)",1


---

In [9]:
data['ttext_clean'] = data['ttext'].apply(lambda t:[ w.strip() for w in t.split() if w.strip() ] )

In [10]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'^http.*',' url ', w.strip() ) for w in t  ]
  )

In [11]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'[:;]-*[)D]',' happysmile ', w.strip() )for w in t ]
  )

In [12]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'\)\)\)*',' happysmile ', w.strip() ) for w in t ]
  )

In [13]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'[:;]\*',' kisssmile ', w.strip() ) for w in t ]
  )

In [14]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r':\(',' sadsmile ', w.strip() ) for w in t ]
  )

In [15]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'\(\(\(*',' sadsmile ', w.strip() ) for w in t ]
  )

In [19]:
data['ttext_clean'] = [ ' '.join(s) for s in data['ttext_clean'] ]

In [20]:
data['ttext_clean'] = data['ttext_clean'].str.lower()
data['ttext_clean'] = data['ttext_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
data['ttext_clean'] = data['ttext_clean'].apply(lambda s: re.sub( r'_', ' ', s))
data['ttext_clean'] = data['ttext_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' digit ', s)) 


In [22]:
data['ttext_clean'] = data['ttext_clean'].apply(lambda t:[ w.strip() for w in t.split() if w.strip() ] )

In [24]:
# замена буквенно-цифровых кодов
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t: [w for w in t if not re.match( r'\b.*\d+.*\b', w) ]
)

In [ ]:
# data[['ttext_clean']]
# data[['ttext']]

---

In [26]:
with gzip.open('../data/text/stop-nltk.txt.gz','rt',encoding='utf-8') as f: 
    stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )
ppr(stopwords)

записей: 151


In [27]:
# удаление лишних слов
data['ttext_clean'] = data['ttext_clean'].apply(lambda t:[w for w in t if w not in stopwords])

In [ ]:
%xdel stopwords

In [33]:
# удаление коротких слов
data['ttext_clean'] = data['ttext_clean'].apply(lambda t:[w for w in t if len(w)>2])

---

In [37]:
# data[ data['ttext_clean'].str.len()<1 ][['ttext_clean']]

In [35]:
ppr(data)
data = data[ data['ttext_clean'].str.len()>0 ].reset_index(drop=True) 
ppr(data)

записей: 226 830
записей: 226 826


In [36]:
data.sample(3)

,id,ttext,ttype,ttext_clean
84502,410785793595428864,Это последний вечер когда я буду портить свой желудок)маман приезжает),1,"[это, последний, вечер, буду, портить, свой, желудок, маман, приезжает]"
91265,410852115385704448,"RT @TomAnnailla: @2014Gayane все норм. твой предыдущий твит меня убил - так, что отхожу по тихоньку:)",1,"[tomannailla, норм, твой, предыдущий, твит, убил, отхожу, тихоньку, happysmile]"
138824,411945714097192960,"это такой пиздец,когда твоя знакомая до лета в калифорнии и не знает,кто такие нбхд и что они тоже там живут\n:""(",-1,"[это, пиздец, твоя, знакомая, лета, калифорнии, знает, такие, нбхд, живут]"


In [98]:
vocab = ['PAD'] + sorted(set([ w for t in data['ttext_clean'] for w in t if w ]))
ppr(vocab)

записей: 239 403


In [100]:
# vocab

In [101]:
vocab = { w:n for n,w in enumerate(vocab) }

In [102]:
data['ttext_code'] = data['ttext_clean'].apply(lambda t: [ vocab[w] for w in t ] )

In [103]:
data['ttext_code'].str.len().describe()

count    226826.000000
mean          8.244011
std           3.150601
min           1.000000
25%           6.000000
50%           8.000000
75%          10.000000
max          30.000000
Name: ttext_code, dtype: float64

In [104]:
n_max = data['ttext_code'].str.len().max()
n_max

30

In [105]:
pad = [0]*n_max

In [106]:
data['ttext_code'] = data['ttext_code'].apply(lambda t: list(reversed(t + pad[len(t):])) )

In [107]:
# data['ttext_code']

In [108]:
X = np.vstack(data['ttext_code'].values)
y = data['ttype'].values

In [109]:
y.shape,X.shape

((226826,), (226826, 30))

In [110]:
# X

---

In [111]:
import tensorflow as tf
from tensorflow.python.client import device_lib

import keras 

# import matplotlib.pyplot as plt
# import os

In [112]:
print(tf.__version__)
print(keras.__version__)
print(device_lib.list_local_devices())

1.13.1
2.2.2
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11124188001171681502
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16169508617117255816
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 314649200904835500
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1359216640
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11356980979617569427
physical_device_desc: "device: 0, name: GeForce GT 730, pci bus id: 0000:01:00.0, compute capability: 3.5"
]


In [113]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

In [114]:
max_features = 100000
maxlen = 30
batch_size = 32

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [115]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              class_mode="binary")

In [116]:
model.fit(
    X, y, 
    batch_size=batch_size, 
    nb_epoch=1 # , show_accuracy=True
)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


ValueError: ('Some keys in session_kwargs are not supported at this time: %s', dict_keys(['class_mode']))

In [ ]:
result = model.predict_proba(X)

---

In [ ]:
print(device_lib.list_local_devices())

In [ ]:
dev = tf.test.gpu_device_name()
print('Default GPU Device:',dev)

In [ ]:
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# # sess.list_devices()

In [ ]:
print(tf.test.is_built_with_cuda())